In [ ]:
import openai
import os
import pytesseract
import cv2
from pdf2image import convert_from_path
import numpy as np 
import random
import time 
import regex as re
import pickle 

In [ ]:
!pip install python-dotenv


In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = "API-KEY"

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [ ]:
def get_examples(samples):
    
    """
    Concatenate the examples into a single string of text, where each example is delimeted by <(< >)>
    
    """
    text = ''  
    i = 1
    for sample in samples:

        text = text + ' Document Example  '+str(i)+'\n <(<  \n' + sample +' >)> \n\n'
        i+=1
    return text

In [ ]:
def get_prompt(text):
    
    prompt = f"""
                You are a fake data generator of different types of insurance policy documents. You will be given examples
                of some insurance documents, you should study the examples and generate similar fake data.
                
                The data generation is done in the following steps:
        
                Step 1: Study all the examples of insurance policy documents given below, each example is delimeted by '<(<'  '>)>' 

                Step 2: Generate a single fake insurance policy document that is similar to the examples but not identical to them.


                Step 3: In each generated document, check if it contains the following information: 
                
                                1. Insurance Company Name
                                2. Policy holder name
                                3. Broker information 
                                4. Policy number 
                                5. Type of insurance
                                6. Contract start date
                                7. Contract termination date 

               Step 4: If the information is found, extract it. If not found, just state "not found" 

               Step 5: The Output should be consist of the generated document as a string and the extracted information in a JSON format.
               
               
                Examples: {text}

              """
    return prompt


In [ ]:
def get_prompt_augment_1(text):
    
    prompt = f"""
                You are a fake data generator of different types of insurance policy documents. You will be given examples
                of some insurance documents, you should study the examples and generate a similar document but with 
                different infotmation.
                
                The data generation is done in the following steps:
        
                Step 1: Study the insurance policy document given, it is delimeted by '<(<, >)>'
                
                Step 2: Output the same document  and the same but change the details in the document and extracted information. 
               
                Document: {text}

              """
    return prompt


In [ ]:
def get_prompt_augment_2(text):
    
    prompt = f"""
                You will be given some insurance policy documents in different languages, where the document is delimeted
                by '<(<' '>)>':
                
                For each document:
                
                    1. Detect the language of the document. 
                    
                    2. Translate it into  a different language that can be either Dutch, Spanish, or French
                    
                    3. Ouput the translated document along with it's extracted information 
               
                Document: {text}

              """
    return prompt

In [ ]:
path = '/Users/mariam/Desktop/KU Leuven Classes/Emergent_CP/policies/' #path to policy samples
custom_config = r'--oem 3 --psm 6'
files = os.listdir(path) #extract all files in the directory of the policy samples
examples = []
for file in files: #In each PDF file, extract the text in the pages and add them to list of examples. 
    pages = convert_from_path(path+file)
    for page in pages: 
        
        img = np.array(page)

        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

        examples.append(pytesseract.image_to_string(img, config=custom_config, lang='eng'))




In [ ]:
data = [] #List where we store the generated fake data
samples_log = [] #List where we add the examples used to generate the data, for troubleshooting purposes
n=4 # number of samples we want 

In [ ]:

n=4
for i in range(40):


    for i in range(n):
        samples = random.sample(examples, 3) #randomly sample 3 examples 
        samples_log.append(samples) #add those example to a log to keep track of which samples where used to generate data.
        train_samples = get_examples(samples) #Concatenate selected samples to a single series of text
        prompt = get_prompt(train_samples) #Feed the samples to get the final prompt
        data.append(get_completion(prompt)) #Give the prompt to ChatGPT
    time.sleep(30)

In [ ]:
d = []
for i in range(50):
    document = random.sample(data[:221],1 )[0]
    d.append(document)
    prompt= get_prompt_augment_1('<(< \n ' + document + '\n >)>')
    data.append(get_completion(prompt))
   
    if i%4==0:
            time.sleep(30)

In [ ]:
t = []
for i in range(50):
    document = random.sample(data,1 )[0]
    t.append(document)
    prompt= get_prompt_augment_2('<(< \n ' + document + '\n >)>')
    data.append(get_completion(prompt))
   
    if i%4==0:
            time.sleep(30)

In [ ]:

with open("data", "wb") as fp:   #Pickling
    pickle.dump(data_split, fp)
import pickle
with open("data", "rb") as fp:   # Unpickling
     b = pickle.load(fp)